In [1]:
import csv
from itertools import product
from pandas import read_excel
import numpy as np
import pandas as pd
import os

# Number of pairs
dataset_size = 7000 


# Give the location of the files
directory = 'datasets_labeled/'

loc_generated_devset = ('datasets/dev_set.csv')
loc_generated_trainset = ('datasets/train_set.csv')
loc_generated_testset = ('datasets/test_set.csv')


In [2]:


def get_all_pairs_from_file(directory, filename):
    term = filename.split('.')[0]
    df_rawgroups = read_excel(directory + filename, sheet_name="groups", index_col=None, header=None, engine='openpyxl')
    df_rawgroups = df_rawgroups.drop(df_rawgroups.columns[[0, 1]], axis=1)  # removing first and third unwanted column
    df_rawgroups = df_rawgroups.drop([0, 1])  # removing first and third unwanted column

    df_group_similarities = read_excel(directory + filename, sheet_name="group similarities", index_col=None, header=None, engine='openpyxl')
    df_group_similarities = df_group_similarities.drop(0, axis=0)
    df_group_similarities = df_group_similarities.drop(0, axis=1)

    df_definition = df_rawgroups[df_rawgroups.columns[range(0, df_rawgroups.shape[1]-1, 4)]]

    groups = []
    for column in df_definition.columns:
        group = []
        for definition in df_definition[column]: 
            if not pd.isna(definition): group.append(definition)
        groups.append(group)    
        pairs = []

    for grp_index1, group1 in enumerate(groups):
        for definition1 in group1:
            for grp_index2, group2 in enumerate(groups):
                for definition2 in group2:
                    if definition1 is not definition2:
                        sim = df_group_similarities.iloc[grp_index1,grp_index2]
                        if type(sim) is str: sim = sim.upper() 
                        if len(str(sim)) == 1 and (grp_index1 <= grp_index2) and not pd.isna(sim): 
                            pairs.append([term, term + str(grp_index1), term + str(grp_index2), definition1, definition2, sim])
    return pairs

pairs = []                  
for filename in os.listdir(directory):
    if not filename.startswith("~") and filename.endswith(".xlsx"):
        print(os.path.join(directory, filename))
        pairs.extend(get_all_pairs_from_file(directory, filename))
    else:
        continue

df_pairs = pd.DataFrame(pairs, columns=["term", "grp_id1", "grp_id2", "definition1", "definition2", "sim"])

df_pairs

datasets_labeled/accuracy.xlsx
datasets_labeled/alarm.xlsx
datasets_labeled/application_process.xlsx
datasets_labeled/asset.xlsx
datasets_labeled/audit.xlsx
datasets_labeled/ball grid array.xlsx
datasets_labeled/bus coupler.xlsx
datasets_labeled/client.xlsx
datasets_labeled/confidentiality.xlsx
datasets_labeled/device.xlsx
datasets_labeled/diversity.xlsx
datasets_labeled/domain.xlsx
datasets_labeled/double_insulation.xlsx
datasets_labeled/earth.xlsx
datasets_labeled/entity.xlsx
datasets_labeled/environment.xlsx
datasets_labeled/event.xlsx
datasets_labeled/extended_link.xlsx
datasets_labeled/fault.xlsx
datasets_labeled/FIRE.xlsx
datasets_labeled/frame.xlsx
datasets_labeled/group.xlsx
datasets_labeled/half-value layer.xlsx
datasets_labeled/hazard.xlsx
datasets_labeled/incident.xlsx
datasets_labeled/integrity.xlsx
datasets_labeled/node.xlsx
datasets_labeled/process.xlsx
datasets_labeled/radiator.xlsx
datasets_labeled/RADIOGRAPHY.xlsx
datasets_labeled/redundancy.xlsx
datasets_labeled/Repla

,term,grp_id1,grp_id2,definition1,definition2,sim
0,accuracy,accuracy1,accuracy2,synonym etc.,too long,A
1,alarm,alarm1,alarm2,Hätten das zwei Gruppen sein sollen?,A criticality accident is an uncontrolled nucl...,D
2,alarm,alarm1,alarm3,Hätten das zwei Gruppen sein sollen?,Ursache,D
3,alarm,alarm1,alarm7,Hätten das zwei Gruppen sein sollen?,Meinen 'condition' und 'state' hier beide Male...,A
4,alarm,alarm1,alarm13,Hätten das zwei Gruppen sein sollen?,"Ein Feldbus ist ein Bussystem, das in einer An...",A
...,...,...,...,...,...,...
8188,_3_dB_bandwidth,_3_dB_bandwidth0,_3_dB_bandwidth0,frequency bandwidth between two points in a fr...,frequency bandwidth between two points in a fr...,1
8189,_3_dB_bandwidth,_3_dB_bandwidth0,_3_dB_bandwidth0,frequency bandwidth between two points in a fr...,frequency bandwidth between two points in a fr...,1
8190,_3_dB_bandwidth,_3_dB_bandwidth0,_3_dB_bandwidth0,frequency bandwidth between two points in a fr...,frequency bandwidth between two points in a fr...,1
8191,_3_dB_bandwidth,_3_dB_bandwidth0,_3_dB_bandwidth0,frequency bandwidth between two points in a fr...,frequency bandwidth between two points in a fr...,1


In [3]:
print(90*'=')
print("Datenset zusammenstellen: Möglichst ausgeglichene Anzahl von similarity labels,\nBenennungen und Begriffsgruppen erstellen")
print(90*'=' + "\n")

df = df_pairs.copy(deep=True)
df.insert(df.shape[1], "count_sim", [0]*df.shape[0], True) 
df.insert(df.shape[1], "count_term", [0]*df.shape[0], True) 
df.insert(df.shape[1], "count_grpid1", [0]*df.shape[0], True) 
df.insert(df.shape[1], "count_grpid2", [0]*df.shape[0], True) 
df.insert(df.shape[1], "count_def", [0]*df.shape[0], True) 

df_dataset = pd.DataFrame(columns=df.columns)

for i in range (0, dataset_size):
    #erstes Paar auswählen und entfernen
    select = df.iloc[0]
    df_dataset = df_dataset.append(select)
    df.drop(select.name, inplace=True, errors='raise')
    
    #eigenschaften hochzählen
    df.loc[df.sim == select.sim, 'count_sim'] += 1
    df.loc[df.term == select.term, 'count_term'] += 1
    df.loc[df.grp_id1 == select.grp_id1, 'count_grpid1'] += 1
    df.loc[df.grp_id1 == select.grp_id2, 'count_grpid2'] += 1
    df.loc[df.grp_id2 == select.grp_id1, 'count_grpid1'] += 1
    df.loc[df.grp_id2 == select.grp_id2, 'count_grpid2'] += 1
    df.loc[df.definition1 == select.definition1, 'count_def'] += 1
    df.loc[df.definition1 == select.definition2, 'count_def'] += 1
    df.loc[df.definition2 == select.definition1, 'count_def'] += 1
    df.loc[df.definition2 == select.definition2, 'count_def'] += 1
    
    #sortieren, so dass Paare mit Eigenschaften, die am seltensten vertreten sind, oben stehen 
    df.sort_values(by=['count_sim','count_def','count_grpid1','count_grpid2','count_term'], inplace=True)

    
#Verteilung ausgeben
print(df_dataset.groupby(['sim']).size())
print('')
print(df_dataset.groupby(['term']).size())




Datenset zusammenstellen: Möglichst ausgeglichene Anzahl von similarity labels,
Benennungen und Begriffsgruppen erstellen

sim
1    1761
        1
A    1143
B    1761
C    1338
D     996
dtype: int64

term
FIRE                         315
RADIOGRAPHY                    2
Replace_message               15
_3_dB_bandwidth                6
accuracy                       1
alarm                         16
asset                          8
audit                          6
ball grid array                4
bus coupler                    2
client                       632
confidentiality               49
device                      1084
domain                       160
earth                         74
entity                        69
environment                  114
event                        417
extended_link                  4
fault                        105
frame                        487
group                         38
half-value layer               6
hazard                        89
in

In [4]:
print(90*'=')
print("Datenset speichern")
print(90*'=' + "\n")
from sklearn.model_selection import train_test_split

testset_terms = ["domain", "hazard", "group", "event"]

train_df = df_dataset[~df_dataset["term"].isin(testset_terms)]
test_df = df_dataset[df_dataset["term"].isin(testset_terms)]
test_df.to_csv(loc_generated_testset)


train, dev = train_test_split(train_df, test_size=0.2)
train.to_csv(loc_generated_trainset)
dev.to_csv(loc_generated_devset)

Datenset speichern



In [ ]:
for term in df_dataset["term"].unique():
    df_dataset[df_dataset["term"] == term].to_csv("datasets_by_term/"+term+".csv")

In [ ]:
test_df

In [ ]:
train_df[train_df["term"]=="client"]